In [3]:
from typing import List
from IPython.display import display, Math, Latex

def bb(n, bits=7):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    return ('{0:0' + str(bits) + 'b}').format(n & mask)
  
def bn(n, bits=7, underline=None, sep=None):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    arr = list(('{0:0' + str(bits) + 'b}').format(n & mask))
    
    arr.reverse()
    if underline is not None: arr[underline] = r'\underline{' + arr[underline] + '}'
    if sep is not None: arr.insert(sep, '|')
    arr.reverse()
    return '\ '.join(arr)
 

def text(s: str) -> str:
  s = s.replace(' ', '\ ')
  s = r'\textup{' + s + '}'
  return s

def let(a: str, sign = 0, sub = '') -> str:
  r = r' \left[' + ('-' if sign else '') + a + r'\right]_{\textup{' + sub + r'}}'
  return r

def arr(l: List[str], t='c') -> str:
  r = r'\begin{array}{' + t + r'}'
  r += r' \\ '.join(l)
  r += r'\end{array}'
  return r

def fix(val, digits=16):
  # return int(bb(val, digits), 2)
  val &= ((1<<digits)-1)
  if val & (1<<(digits - 1)): val -= (1<<digits)
  return val

def sign(n: int):
  if n < 0: return 1
  return 0

class Mul2():
  table: List[List[str]]
  
  def __init__(self, a: int, b: int) -> None:
    self.a = a
    self.b = b
    self.table = [[
        arr([
          text('Оперaнды'),
        ]),
        arr([
          text('СЧП (стaршие рaзряды)'),
        ]),
        arr([
          text(' В/СЧП (млaдшие рaзряды)'),
        ]),
        arr([
          text('Признaк коррекции'),
        ])
    ]]
    self.top = ''
    self.bot = ''
    self.process(a, b)
  
  def __str__(self) -> str:
    return self.print()
    
  def print(self):
    la = r'[A]_{пр.}' if self.a > 0 else  r'[-A]_{доп.}'
    lb = r'[B]_{пр.}' if self.b > 0 else  r'[-B]_{доп.}'
    la += '=' + bb(self.a)
    lb += '=' + bb(self.b)
    self.top = (
      '(A' + ('>' if self.a > 0 else '<') + '0,\ B' + ('>' if self.b > 0 else '<') + '0)')
    self.top += r' \\ ' + '\n'
    self.top += '\left.' + la + ';\ ' + lb + '\\right.'
    
    res = r'\begin{array}{c}'
    # res += self.top
    # res += r' \\ '  + '\n' + r' \\ '
    res += (
      r'\begin{array}{|c|c|c|c|} \hline ' +
      (r' \\ \hline ' + '\n').join(' & '.join(row) for row in self.table) +
      r' \\ \hline ' + '\n' + r' \end{array} \\' + '\n'
      )
    res += r' \\ ' + '\n' + r' \\ '
    res += self.bot
    res += r'\end{array}'
    
    return f'$${res}$$'
    # display(Math(res))
    
  def process(self, a: int, b: int):
    s = b
    self.table.append([
      text('СЧП'),
      bb(s >> 12, 17),
      bb(s, 12),
      str(0)
    ])
    cor = 0
    for i in range(3):
      t = []
      r = bb(s, 12)[-4:]
      ss = 0
      l = ''
      print(r)

      if   r == '0000':s1,s2,l1,l2,cor = 0*a  ,+ 0 * a, '0A ' ,'+0A',0                 
      elif r == '0001':s1,s2,l1,l2,cor = 0*a  ,+ 1 * a, '0A ' ,'+1A',0           
      elif r == '0010':s1,s2,l1,l2,cor = 0*a  ,+ 2 * a, '0A ' ,'+2A',0           
      elif r == '0011':s1,s2,l1,l2,cor = 4*a  ,-1*a , '4A ' ,'-A ',0          
      elif r == '0100':s1,s2,l1,l2,cor = 4*a  ,+ 0 * a, '4A ' ,'+0A',0           
      elif r == '0101':s1,s2,l1,l2,cor = 4*a  ,+ 1 * a , '4A ' ,'+A ',0          
      elif r == '0110':s1,s2,l1,l2,cor = 4*a  ,+ 2 * a, '4A ' ,'+2A',0           
      elif r == '0111':s1,s2,l1,l2,cor = 8*a  ,- 1 * a , '8A ' ,'-A ',0          
      elif r == '1000':s1,s2,l1,l2,cor = 8*a  ,+ 0 * a, '8A ' ,'+0A',0           
      elif r == '1001':s1,s2,l1,l2,cor = 8*a  ,+ 1 * a , '8A ' ,'+A ',0          
      elif r == '1010':s1,s2,l1,l2,cor = 8*a  ,+ 2 * a, '8A ' ,'+2A',0           
      elif r == '1011':s1,s2,l1,l2,cor = -4*a ,- 1 * a , '-4A' ,'-A ',1           
      elif r == '1100':s1,s2,l1,l2,cor = -4*a ,+ 0 * a, '-4A' ,'+0A',1            
      elif r == '1101':s1,s2,l1,l2,cor = -4*a ,+ 1 * a , '-4A' ,'+A ',1           
      elif r == '1110':s1,s2,l1,l2,cor = -4*a ,+ 2 * a, '-4A' ,'+2A',1            
      elif r == '1111':s1,s2,l1,l2,cor = 0*a  ,- 1 * a	, '0A ' ,'-A ',1            

      sss = s + (s1 << 12)+ (s2 << 12)
      self.table.append([
        arr([
          let(l1),
          let(l2),
          text('СЧП'),
          text('СЧП \\rightarrow 4'),
        ]),
        arr([
          bb((s1    ), 17),
          bb((s2    ), 17),
          bb(sss >> 12, 17),
          bb(sss >> 16, 17)
        ]),
        arr([
          '',
          '',
          bb(sss, 12),
          bb(sss >> 24, 12)
        ]),
        str(cor)
      ])
      s = sss
      s = s >> 4
    
    print(bin(int(bb(s >> 12, 12), 2)))
    # self.bot = r' \left[C\right]_{\textup{пр.}}' if a > 0 else  r' \left[C\right]_{\textup{доп}.}'
    # self.bot += '='
    # self.bot += bb( a * b, aw * 2) + '_2 = '
    # self.bot += str(a * b) + '_{10}'
    
open('res4.tex', 'w+', encoding='utf-8').write(f'''
  {Mul2(0b100011100110, (0b100100010110))}
''')
# open('res.tex', 'w+', encoding='utf-8').write(f'''
#   {Mul2(0b011101001101, 0b100011001101)}
# ''')
0b010000000001


0110
0001
1001
0b10100001101


1025